# PCA as a Factor Model - Coding Exercises


### Introduction

As we learned in the previous lessons, we can use PCA to create a factor model of risk. Our risk factor model represents the return as:

$$
\textbf{r} = \textbf{B}\textbf{f} + \textbf{s}
$$

where $\textbf{r}$ is a matrix containing the asset returns, $\textbf{B}$ is a matrix representing the factor exposures, $\textbf{f}$ is the matrix of factor returns, and $\textbf{s}$ is the idiosyncratic risk (also known as the company specific risk).

In this notebook, we will use real stock data to calculate:

* The Factor Exposures (Factor Betas) $\textbf{B}$
* The Factor Returns $\textbf{f}$
* The Idiosyncratic Risk Matrix $\textbf{S}$
* The Factor Covariance Matrix $\textbf{F}$

We will then combine these quantities to create our Risk Model. 

### Install Packages

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

# Get Returns

In this notebook, we will get the stock returns using Zipline and data from Quotemedia, just as we learned in previous lessons. The function `get_returns(start_date, end_date)` in the `utils` module, gets the data from the Quotemedia data bundle and produces the stock returns for the given `start_date` and `end_date`. You are welcome to take a look at the `utils` module to see how this is done.

In the code below, we use `utils.get_returns` funtion to get the returns for stock data between `2011-01-05` and `2016-01-05`. You can change the start and end dates, but if you do, you have to make sure the dates are valid trading dates. 

In [2]:
import utils

# Get the returns for the fiven start and end date. Both dates must be valid trading dates
returns = utils.get_returns(start_date='2011-01-05', end_date='2016-01-05')

# Display the first rows of the returns
returns.head()

,Equity(0 [A]),Equity(1 [AAL]),Equity(2 [AAP]),Equity(3 [AAPL]),Equity(4 [ABBV]),Equity(5 [ABC]),Equity(6 [ABT]),Equity(7 [ACN]),Equity(8 [ADBE]),Equity(9 [ADI]),...,Equity(481 [XL]),Equity(482 [XLNX]),Equity(483 [XOM]),Equity(484 [XRAY]),Equity(485 [XRX]),Equity(486 [XYL]),Equity(487 [YUM]),Equity(488 [ZBH]),Equity(489 [ZION]),Equity(490 [ZTS])
2011-01-07 00:00:00+00:00,0.008437,0.014230,0.026702,0.007146,0.0,0.001994,0.004165,0.001648,-0.007127,-0.005818,...,-0.001838,-0.005619,0.005461,-0.004044,-0.013953,0.0,0.012457,-0.000181,-0.010458,0.0
2011-01-10 00:00:00+00:00,-0.004174,0.006195,0.007435,0.018852,0.0,-0.005714,-0.008896,-0.008854,0.028714,0.002926,...,0.000947,0.007814,-0.006081,0.010466,0.009733,0.0,0.001440,0.007784,-0.017945,0.0
2011-01-11 00:00:00+00:00,-0.001886,-0.043644,-0.005927,-0.002367,0.0,0.009783,-0.002067,0.013717,0.000607,0.008753,...,0.001314,0.010179,0.007442,0.007351,0.006116,0.0,-0.006470,0.035676,0.007467,0.0
2011-01-12 00:00:00+00:00,0.017254,-0.008237,0.013387,0.008133,0.0,-0.005979,-0.001011,0.022969,0.017950,0.000257,...,0.004986,0.015666,0.011763,0.027182,0.004386,0.0,0.002631,0.014741,-0.011903,0.0
2011-01-13 00:00:00+00:00,-0.004559,0.000955,0.003031,0.003657,0.0,0.014925,-0.004451,-0.000400,-0.005719,-0.005012,...,0.030499,-0.003217,0.001694,0.000547,-0.018235,0.0,-0.005084,-0.004665,-0.009178,0.0


In [3]:
returns.index

DatetimeIndex(['2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14', '2011-01-18', '2011-01-19',
               '2011-01-20', '2011-01-21',
               ...
               '2015-12-21', '2015-12-22', '2015-12-23', '2015-12-24',
               '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31',
               '2016-01-04', '2016-01-05'],
              dtype='datetime64[ns, UTC]', length=1256, freq='C')

In [4]:
returns.columns

Index([     Equity(0 [A]),    Equity(1 [AAL]),    Equity(2 [AAP]),
         Equity(3 [AAPL]),   Equity(4 [ABBV]),    Equity(5 [ABC]),
          Equity(6 [ABT]),    Equity(7 [ACN]),   Equity(8 [ADBE]),
          Equity(9 [ADI]),
       ...
         Equity(481 [XL]), Equity(482 [XLNX]),  Equity(483 [XOM]),
       Equity(484 [XRAY]),  Equity(485 [XRX]),  Equity(486 [XYL]),
        Equity(487 [YUM]),  Equity(488 [ZBH]), Equity(489 [ZION]),
        Equity(490 [ZTS])],
      dtype='object', length=490)

In [5]:
returns.shape

(1256, 490)

# TODO: Factor Exposures

In the code below, write a function, `factor_betas(pca, factor_beta_indices, factor_beta_columns)` that calculates the factor exposures from Scikit-Learn's `PCA()` class. Remember the matrix of factor exposures, $\textbf{B}$, describes the coordintates of the Principal Components in the original basis. The `pca` parameter must be a Scikit-Learn's pca object, that has fit the model with the returns. In other words, you must first run `pca.fit(returns)` before passing this parameter into the function. Later in this notebook we will create a function, `fit_pca()`, that will fit the pca model and return the `pca`object. The `factor_beta_indices` parameter must be a 1 dimensional ndarray containg the column names of the `returns` dataframe. The `factor_beta_columns` parameter must be a 1 dimensional ndarray containing evenly spaced integers from 0 up to the number of principal components you used in your `pca` model minus one. For example, if you used 5 principal compoenents in your `pca` model, `pca = PCA(n_components = 5)`, then `factor_beta_columns = [0, 1, 2, 3, 4]`. This function has to return a Pandas dataframe with the factor exposures, where the `factor_beta_indices` correspond to the indices of the dataframe and the `factor_beta_columns` correspond to the column names of the dataframe. 

### UNDERSTANDING NOTE: 
**pca.components_.shape = factors * num_stocks**

e.g. 
<br>returns.shape = 1256 * 490 (1256 different time points * 490 stocks)
<br>$\textbf{N}$ = Number of companes = 490
<br>$\textbf{K}$ = Number of factors = 20
<br>$\textbf{T}$ = Number of time points 1256

pca = PCA(K) `top K factors`

pca.fit(returns) `fits the returns matrix into K_factors
after this method components_ has factor exposures aka beta in shape K,N`

Requirement
<br>$\textbf{R}$[N,T] = $\textbf{B}$[N,K]*$\textbf{F}$[K,T] + $\textbf{S}$[N,T]

pca.components_.shape = 20 , 490 `i.e. K,N`



In [44]:
def factor_betas(pca, factor_beta_indices, factor_beta_columns):

    #Implement Function
    
    #The factor_beta_indices parameter must be a 1 dimensional ndarray containg the 
    #column *names* of the returns dataframe.
    
    ##The factor_beta_columns parameter must be a 1 dimensional ndarray containing evenly spaced integers 
    ##from 0 up to the number of principal components you used in your pca model minus one. 
    
    # pca.components_: array, shape (n_components(K), n_features(N))
    # required shape: 𝐁[N,K]
    
    B = pd.DataFrame(data = pca.components_.T, index=factor_beta_indices, columns=factor_beta_columns)
    return B


# TODO: Factor Retuns

In the code below, write a function, `factor_returns(pca, returns, factor_return_indices, factor_return_columns)` that calculates the factor returns from Scikit-Learn's `PCA()` class. Remember the matrix of factor returns, $\textbf{f}$, represents the `returns` written in the **new** basis. The `pca` parameter must be a Scikit-Learn's pca object, that has fit the model with the returns. In other words, you must first run `pca.fit(returns)` before passing this parameter into the function. Later in this notebook we will create a function, `fit_pca()`, that will fit the pca model and return the `pca`object. The `returns` parameter is the pandas dataframe of returns given at the begining of the notebook. The `factor_return_indices` parameter must be a 1 dimensional ndarray containing the trading dates (Pandas `DatetimeIndex`) in the `returns` dataframe. The `factor_return_columns` parameter must be a 1 dimensional ndarray containing evenly spaced integers from 0 up to the number of principal components you used in your `pca` model minus one. For example, if you used 5 principal compoenents in your `pca` model, `pca = PCA(n_components = 5)`, then `factor_beta_columns = [0, 1, 2, 3, 4]`. This function has to return a Pandas dataframe with the factor returns, where the `factor_return_indices` correspond to the indices of the dataframe and the `factor_return_columns` correspond to the column names of the dataframe.

In [56]:
import pandas as pd 

def factor_returns(pca, returns, factor_return_indices, factor_return_columns):
    
    #Implement Function
    
    #The factor_return_indices parameter must be a 1 dimensional ndarray 
    #containing the trading dates (Pandas DatetimeIndex) in the returns dataframe
    
    ##The factor_return_columns parameter must be a 1 dimensional ndarray 
    ##containing evenly spaced integers from 0 up to the 
    ##number of principal components you used in your pca model minus one
    
    # pca.transform(returns).shape:  (T, K) e.g. (1256, 20) i.e. (Time, Factors)
    
    f = pd.DataFrame(data = pca.transform(returns), 
                    index = factor_return_indices, 
                    columns=factor_return_columns)
    return f


# TODO: Idiosyncratic Risk Matrix

Let's review how we can calculate the Idiosyncratic Risk Matrix $\textbf{S}$. We know that: 

$$
\textbf{s} = \textbf{r} - \textbf{B}\textbf{f}
$$

We refer to $\textbf{s}$ as the residuals. To calculate the idiosyncratic or specific risk matrix $\textbf{S}$, we have to calculate the covariance matrix of the residuals, $\textbf{s}$, and set the off-diagonal elements to zero. 

With this in mind, in the code below cerate a function, `idiosyncratic_var_matrix(returns, factor_returns, factor_betas, ann_factor)` that calclates the **annualized** Idiosyncratic Risk Matrix. The `returns` parameter is the pandas dataframe of returns given at the begining of the notebook. The `factor_returns` parameter is the output of the `factor_returns()` function created above. Similarly, the `factor_betas` parameter is the output of the `factor_betas()` function created above. The `ann_factor` parameter is an integer representing the annualization factor. 

Remember that if the `returns` time series are daily returns, then when we calculate the Idiosyncratic Risk Matrix we will get values on a daily basis. We can annualize these values simply by multiplying the whole Idiosyncratic Risk Matrix by an annualization factor of 252. Remember we don't need the square root of the factor because our numbers here are variances not standard deviations.

The function must return a pandas dataframe with the annualized Idiosyncratic Risk Matrix containing the covariance of the residuals in its main diagonal and with all the off-diagonal elements set to zero. 

In [50]:
def idiosyncratic_var_matrix(returns, factor_returns, factor_betas, ann_factor):
    
    #Implement Function
    #r.shape = T,N i.e. 1256, 490
    #B.shape = N,K i.e. 490, 20
    #f.shape = T,K e.g. 1256, 20
    #𝐬=𝐫−𝐁𝐟
    #s = r(T,N) - f(T,K)* B(N,K).T
    common_returns_ = pd.DataFrame(np.dot(factor_returns, factor_betas.T), returns.index, returns.columns)
    residuals_ = (returns - common_returns_)

    return pd.DataFrame(np.diag(np.var(residuals_))*ann_factor, returns.columns, returns.columns)    


# TODO: Factor Covariance Matrix

To calculate the annualized factor covariance matrix, $\textbf{F}$, we use the following equation:

$$
\textbf{F} = \frac{1}{N -1}\textbf{f}\textbf{f}^T
$$

where, $N$ is the number of elements in $\textbf{f}$. Recall that the factor covariance matrix, $\textbf{F}$, is a diagonal matrix.

With this in mind, create a function, `factor_cov_matrix(factor_returns, ann_factor)` that calculates the annualized factor covariance matrix from the factor returns $\textbf{f}$. The `factor_returns` parameter is the output of the `factor_returns()` function created above and the `ann_factor` parameter is an integer representing the annualization factor. The function must return a diagonal numpy ndarray 

**HINT :** You can calculate the factor covariance matrix $\textbf{F}$ very easily using Numpy's `.var` method. The $\frac{1}{N -1}$ factor can be taken into account using the `ddof` keyword. 

In [12]:
def factor_cov_matrix(factor_returns, ann_factor):
 
    #Implement Function
    F = np.diag(np.var(factor_returns, ddof=1)*ann_factor)
    return F
    


# TODO: Perfom PCA

In the code below, create a function, `fit_pca(returns, num_factor_exposures, svd_solver)` that uses Scikit-Learn's `PCA()` class to fit the `returns` dataframe with the given number of `num_factor_exposures` (Principal Components) and with the given `svd_solver`. The `returns` parameter is the pandas dataframe of returns given at the begining of the notebook. The `num_factor_exposures` parameter is an integer representing the number of Principal Components you want to use in your PCA algorithm. The `svd_solver` parameter is a string that determines the type of solver you want to use in your PCA algorithm. To see the type of solvers that you can use, see the [Scikit-Learn documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). The function must fit the `returns` and return the `pca` object. 

In [13]:
from sklearn.decomposition import PCA

def fit_pca(returns, num_factor_exposures, svd_solver):

    #TODO: Implement function
    pca = PCA(n_components=num_factor_exposures,svd_solver=svd_solver)
    return pca.fit(returns)


# TODO: Create The Risk Model


In the code below, create a class:

```python
class RiskModel(object):
    def __init__(self, returns, ann_factor, num_factor_exposures, pca):
```

where the `returns` parameter is the pandas dataframe of returns given at the begining of the notebook. The `ann_factor` parameter is an integer representing the annualization factor. The `num_factor_exposures` parameter is an integer representing the number of Principal Components you want to use in your PCA algorithm. The `pca` parameter is the output of the `fit_pca()` function created above. The class must contain all the fucntions created above. For example, to include the Factor covariance matrix we will use:

```python
self.factor_cov_matrix_ = factor_cov_matrix(self.factor_returns_, ann_factor)
```

In [57]:
import numpy as np

class RiskModel(object):
    #Implement class
    def __init__(self, returns, ann_factor, num_factor_exposures, pca):
        
        self.factor_betas_ = factor_betas(pca,
                                          returns.columns.values,
                                          np.arange(num_factor_exposures))
        
        self.factor_returns_ = factor_returns(pca, 
                                              returns, 
                                              returns.index, 
                                              np.arange(num_factor_exposures))
        
        self.idiosyncratic_var_matrix_ = idiosyncratic_var_matrix(returns, 
                                                                  self.factor_returns_, 
                                                                  self.factor_betas_, 
                                                                  ann_factor)
        
        self.factor_cov_matrix_ = factor_cov_matrix(self.factor_returns_, ann_factor)


# Set the annualized factor
ann_factor = 252

# Set the number of factor exposures (principal components) for the PCA algorithm
num_factor_exposures = 20

# Set the svd solver for the PCA algorithm
svd_solver = 'full'

# Fit the PCA Model using the fit_pca() fucntion 
pca = fit_pca(returns, num_factor_exposures, svd_solver)
#print(pca.components_.shape)
#print('pca.transform(returns).shape: ', pca.transform(returns).shape)

# Create a RiskModel object
rm = RiskModel(returns, ann_factor, num_factor_exposures, pca)

# TODO: Print The Factor Exposures

In [58]:
# Display the Factor Exposures
rm.factor_betas_

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Equity(0 [A]),-0.043168,0.019551,-0.009934,0.010540,-0.018198,0.010745,0.001129,0.039739,-0.013805,0.012737,-0.023840,-0.001417,0.020993,0.034610,-0.030489,0.033606,-0.011429,-0.011743,0.005269,-0.005353
Equity(1 [AAL]),-0.058745,0.196377,0.078688,0.082096,0.348478,-0.138082,0.071293,0.105248,-0.045890,-0.085478,0.003283,0.105343,0.067216,-0.044060,-0.022316,0.017263,-0.033108,0.013981,-0.046807,-0.015991
Equity(2 [AAP]),-0.034333,0.034515,0.011338,-0.025437,-0.008172,-0.013183,-0.060952,0.006549,-0.032654,-0.013515,0.028967,0.038365,-0.049605,-0.005246,-0.021970,0.029767,-0.011636,0.051440,0.057169,0.084150
Equity(3 [AAPL]),-0.034100,-0.001393,0.039467,-0.017213,-0.030470,-0.017571,0.023251,0.011430,-0.026568,-0.007719,0.026863,0.007319,0.004818,0.012886,0.052565,-0.073445,-0.031792,0.090794,-0.011189,0.042914
Equity(4 [ABBV]),-0.018031,0.025682,0.004352,-0.070782,0.013199,0.054277,-0.015678,0.040462,-0.012455,-0.035151,-0.051324,-0.040416,-0.051203,-0.018582,-0.011269,-0.042007,0.015886,0.045986,0.015993,0.065880
Equity(5 [ABC]),-0.028900,0.032592,-0.007421,-0.033552,-0.011521,0.026484,0.010850,-0.005704,0.013548,-0.029679,-0.031618,0.017152,-0.003050,0.026356,0.007879,0.008331,-0.018346,-0.013914,0.010620,-0.027396
Equity(6 [ABT]),-0.029057,0.029778,-0.029709,-0.035743,-0.011574,0.060201,-0.020040,0.040985,-0.018040,-0.006208,-0.031578,-0.023080,-0.014368,-0.000543,-0.019329,0.014288,-0.039381,0.010712,0.017120,0.002107
Equity(7 [ACN]),-0.043377,0.002569,0.004132,-0.003493,-0.054307,0.005374,0.002780,0.030432,-0.018802,-0.001783,-0.035404,-0.010932,0.026112,-0.012331,0.003221,0.017700,-0.002851,-0.012359,-0.014009,-0.038557
Equity(8 [ADBE]),-0.047303,0.026612,0.030571,-0.021147,-0.048388,-0.007087,0.007434,0.007471,0.014871,-0.001534,-0.012384,-0.006674,0.035007,-0.043366,0.010603,-0.006379,0.021273,-0.036388,0.001268,-0.048446
Equity(9 [ADI]),-0.047123,-0.003812,0.056008,-0.015538,-0.069462,-0.005604,0.100704,0.063077,0.001608,0.019130,0.011017,0.003487,-0.054657,-0.042238,-0.000143,-0.048432,0.003743,0.045833,0.014835,0.007686


# TODO: Print The Factor Returns

In [ ]:
# Display the Factor Returns


# TODO: Print The Idiosyncratic Risk Matrix

In [ ]:
# Display the Idiosyncratic Risk Matrix


# TODO: Print The Factor Covariance Matrix

In [ ]:
# Display the Factor Covariance Matrix


# View The Percent of Variance Explained by Each Factor

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

# Set the default figure size
plt.rcParams['figure.figsize'] = [10.0, 6.0]

# Make the bar plot
plt.bar(np.arange(num_factor_exposures), pca.explained_variance_ratio_);

You can see that the first factor dominates. The precise defintion of each factor in a latent model is unknown, however we can guess at the likely intepretation.

# View The Factor Returns

Remember that the factors returns don't necessarily have direct interpretations in the real world but you can thinik of them as returns time series for some kind of latent or unknown driver of return variance. 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

# Set the default figure size
plt.rcParams['figure.figsize'] = [10.0, 6.0]

rm.factor_returns_.loc[:,0:5].cumsum().plot();

## Solution

[Solution notebook](pca_factor_model_solution.ipynb)